In [1]:
from astropy import coordinates as coord
from astropy import units as u
from astropy.coordinates import Angle
from gala.coordinates import MagellanicStreamNidever08
import numpy as np
import sys
import pandas as pd
import pyregion


In [52]:
#regfileloc = 'GASKAP.reg'
regfileloc = 'croppedtilefield_old.reg'
importreg= pd.read_csv(regfileloc, delim_whitespace=True, header=None)
regfile=importreg.to_numpy(dtype='str')


In [53]:
regfile

array([['j2000;polygon', '333.149154', '-49.639543', ..., '-44.255155',
        '333.149154', '-49.639543'],
       ['j2000;text', '333.338431', '-46.729577', ..., 'nan', 'nan',
        'nan'],
       ['j2000;polygon', '332.01433', '-55.005671', ..., '-49.651782',
        '332.01433', '-55.005671'],
       ...,
       ['j2000;text', '8.836063000000003', '39.098351', ..., 'nan',
        'nan', 'nan'],
       ['j2000;polygon', '9.569075', '31.359847', ..., '36.73508',
        '9.569075', '31.359847'],
       ['j2000;text', '9.368447', '33.80018', ..., 'nan', 'nan', 'nan']],
      dtype='<U19')

In [54]:
tile_ranges = pd.read_csv('pilot_tile_range.csv')
tile_ranges

,min_l,num_l,max_l,min_b,max_b,num_b
0,-15,1,-10,-10,-3,2.4
1,-10,2,-5,-8,-5,1.6


In [55]:
def find_ms_b_range(tile_ranges, ms_l):
    for row in tile_ranges:
        if row[0] <= ms_l and row[2] >= ms_l:
            #print (row[0], row[2], row[3], row[4])
            return row[3], row[4]
    return None
        
find_ms_b_range(tile_ranges.to_numpy(), -15)

(-10.0, -3.0)

In [56]:
np_tile_ranges = tile_ranges.to_numpy()
clippedarray=np.zeros(11)
#convert from gal to mag and check elements
for i in range(0,len(regfile),2):
    #load in coordinates as j2000, references one of the tile vertices i think,should change to centre where the text is placed
    c = coord.FK5(ra=float(regfile[i+1,1])*u.deg, dec=float(regfile[i+1,2])*u.deg)
    #convert to mag coords
    ms = c.transform_to(MagellanicStreamNidever08)
    # Find the appropriate range
    ms_b_range = find_ms_b_range(np_tile_ranges, ms.L.value)
    if ms_b_range and ms_b_range[0] <= ms.B.value <= ms_b_range[1]:
        print ("tile is inside at L={:.2f} B={:.2f}".format(ms.L.value, ms.B.value))
        clippedarray=np.vstack((clippedarray,regfile[i]))
        clippedarray=np.vstack((clippedarray,regfile[i+1]))
        clippedarray[-1,-5:] = ""

# Remove the first, template row
clippedarray=clippedarray[1:,]


tile is inside at L=-13.95 B=-8.97
tile is inside at L=-13.57 B=-4.71
tile is inside at L=-8.01 B=-5.24


In [57]:

np.savetxt('pilottilefield_simple.reg',clippedarray,fmt='%s')
np.savetxt('pilottilefield_notext_simple.reg',clippedarray[0::2],fmt='%s')

array([['j2000;polygon', '333.149154', '-49.639542999999996', ...,
        '-44.255155', '333.149154', '-49.639542999999996'],
       ['j2000;polygon', '332.01433', '-55.005671', ..., '-49.651782',
        '332.01433', '-55.005671'],
       ['j2000;polygon', '340.189707', '-54.44555', ..., '-49.047952',
        '340.189707', '-54.44555'],
       ...,
       ['j2000;polygon', '119.166896', '-51.313959999999994', ...,
        '-54.112127', '119.166896', '-51.313959999999994'],
       ['j2000;polygon', '8.981391', '36.589022', ..., '41.976765',
        '8.981391', '36.589022'],
       ['j2000;polygon', '9.569075', '31.359847', ...,
        '36.735079999999996', '9.569075', '31.359847']], dtype='<U32')

In [15]:
help(r[0])

Help on Shape in module pyregion.parser_helper object:

class Shape(builtins.object)
 |  Shape.
 |  
 |  Parameters
 |  ----------
 |  shape_name : str
 |      Shape name
 |  shape_params : list
 |      List of parameters
 |  
 |  Examples
 |  --------
 |  >>> import pyregion
 |  >>> region_string = 'fk5;circle(290.96388,14.019167,843.31194")'
 |  >>> shape_list = pyregion.parse(region_string)
 |  >>> shape = shape_list[0]
 |  >>> print(shape.__dict__)
 |  {'attr': ([], {}),
 |   'comment': None,
 |   'continued': None,
 |   'coord_format': 'fk5',
 |   'coord_list': [290.96388, 14.019167, 0.23425331666666666],
 |   'exclude': False,
 |   'name': 'circle',
 |   'params': [Number(290.96388), Number(14.019167), Ang(843.31194")]}
 |  
 |  Methods defined here:
 |  
 |  __init__(self, shape_name, shape_params)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  set_exclude(self)
 |  
 |  -------------------------

In [24]:
c = coord.SkyCoord('00:58:45.255','-72:33:47.83', unit=(u.hourangle,u.deg))
ms_old = c.transform_to(MagellanicStreamNidever08)
print ("{} = {}".format(c, ms_old))
c = coord.SkyCoord('01:04:24.613', '-72:42:33.92', unit=(u.hourangle,u.deg))
ms_new = c.transform_to(MagellanicStreamNidever08)
print ("{} = {}".format(c, ms_new))
diff_l = ms_old.L.value - ms_new.L.value
diff_b = ms_old.B.value - ms_new.B.value
print ("diff is L={:.5f}, B={:.5f}, new pos = '{:.6f},{:.6f}'".format(diff_l, diff_b, 1+diff_b, -2+diff_b))

<SkyCoord (ICRS): (ra, dec) in deg
    (14.6885625, -72.56328611)> = <SkyCoord (MagellanicStreamNidever08): (L, B) in deg
    (-15.58984577, -11.06831348)>
<SkyCoord (ICRS): (ra, dec) in deg
    (16.10255417, -72.70942222)> = <SkyCoord (MagellanicStreamNidever08): (L, B) in deg
    (-15.27212961, -10.74880202)>
diff is L=-0.31772, B=-0.31951, new pos = '0.680489,-2.319511'


In [21]:
cms = coord.SkyCoord(344, -11, MagellanicStreamNidever08, unit=u.deg)
cms.transform_to(coord.FK5).to_string('hmsdms')

'00h57m22.5329s -72d10m06.7132s'